In [1]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = list(glob.glob('../features/*.pkl'))
for l in sorted(fl):
    print("'{}',".format(l))
    
# no other feat7 

'../features/cnn_1_aug1_feat.pkl',
'../features/cnn_1_aug_1_fold8_feat.pkl',
'../features/cnn_1_aug_rescale_preprocess_feat.pkl',
'../features/cnn_1_aug_skimage_denoise_feat.pkl',
'../features/cnn_1_aug_skimage_preprocess_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_2_aug_1_fold8_feat.pkl',
'../features/cnn_2_aug_denoise_preprocess_feat.pkl',
'../features/cnn_2_aug_rescale_preprocess_feat.pkl',
'../features/cnn_2_aug_skimage_preprocess_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_3_aug_1_fold8_feat.pkl',
'../features/cnn_3_aug_denoise_preprocess_feat.pkl',
'../features/cnn_3_aug_rescale_preprocess_feat.pkl',
'../features/cnn_3_aug_skimage_preprocess_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_4_aug_1_fold8_feat.pkl',
'../features/cnn_4_aug_denoise_preprocess_feat.pkl',
'../features/cnn_4_aug_rescale_preprocess_feat.pkl',
'../fea

In [2]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 86) (8424, 86) (1604,)


In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False,verbose_cnt=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 0.85,
                
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=20,
                        verbose_eval=verbose_cnt
                        )
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [4]:
# find new rnd seed
# for i in range(50):
#     rnd = i*6
#     print(rnd)
#     cv_test(3,rnd,False)

In [5]:
# find new rnd seed
# for i in range(50):
#     rnd = i*6
#     print(rnd)
#     cv_test(5,rnd,False)

In [6]:
cv_test(3,42,True,500)

[0]	train-logloss:0.637147	valid-logloss:0.63954
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[123]	train-logloss:0.036986	valid-logloss:0.127287

[0]	train-logloss:0.635534	valid-logloss:0.639735
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[107]	train-logloss:0.041284	valid-logloss:0.140757

[0]	train-logloss:0.636413	valid-logloss:0.639591
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[97]	train-logloss:0.046686	valid-logloss:0.147803

local average valid loss 0.14003944576 val loss std 0.00959489791972
         id  is_iceberg
0  5941774d    0.021510
1  4023181e    0.724113
2  b20200e4    0.0323

In [7]:
cv_test(3,252,True,500)

[0]	train-logloss:0.637027	valid-logloss:0.639504
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[114]	train-logloss:0.040365	valid-logloss:0.126506

[0]	train-logloss:0.635987	valid-logloss:0.638127
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[67]	train-logloss:0.065923	valid-logloss:0.141593

[0]	train-logloss:0.635454	valid-logloss:0.641236
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[98]	train-logloss:0.046232	valid-logloss:0.143883

local average valid loss 0.139547295911 val loss std 0.00780115677448
         id  is_iceberg
0  5941774d    0.028872
1  4023181e    0.572236
2  b20200e4    0.021

In [8]:
cv_test(5,66,True,500)

[0]	train-logloss:0.636709	valid-logloss:0.639726
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[155]	train-logloss:0.03104	valid-logloss:0.117334

[0]	train-logloss:0.635561	valid-logloss:0.640207
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[95]	train-logloss:0.051572	valid-logloss:0.15276

[0]	train-logloss:0.636661	valid-logloss:0.637856
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[75]	train-logloss:0.06601	valid-logloss:0.12872

[0]	train-logloss:0.636992	valid-logloss:0.64087
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logl